# Proyek Capstone: AI-Powered Interview Assessment System
**Tim A25-CS358**

- **Muhammad Rayhan**, M262D5Y1357, sebagai PIC Model & Training (Streamlit/Interface)
- **Hafiz Putra Mahesta**, M262D5Y0714, sebagai PIC Integrasi,Model STT, & Fitur (Confidence Score)
- **Fahri Rasyidin**, M262D5Y0566, sebagai PIC Data & Evaluasi (Dataset, Kunci Jawaban, WER)

## Import Packages/Library yang Digunakan

In [ ]:
!pip install git+https://github.com/openai/whisper.git
!pip install jiwer
!pip install moviepy librosa soundfile
!pip install streamlit pyngrok

In [ ]:
import os
import json
import time
import shutil
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import streamlit as st
import moviepy.editor as mp
import librosa
import soundfile as sf
import whisper
import jiwer
import torch
from datetime import datetime
from tqdm.notebook import tqdm
from pyngrok import ngrok, conf
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

# Load Model dan Data

In [ ]:
BASE_DIR = "/content/drive/MyDrive/Dataset"
VIDEO_INPUT_DIR = os.path.join(BASE_DIR, "Video")
AUDIO_OUTPUT_DIR = os.path.join(BASE_DIR, "Audio")
GROUND_TRUTH_FILE = os.path.join(BASE_DIR, "Transkrip_Manual")

# Cek Folder Video
if os.path.exists(VIDEO_INPUT_DIR):
    video_files = [f for f in os.listdir(VIDEO_INPUT_DIR) if f.lower().endswith(('.mp4', '.webm', '.avi', '.mov', '.mkv'))]
    print(f"Folder Video ditemukan.")
    print(f"Jumlah video yang siap diproses: {len(video_files)} file")
else:
    print(f"Folder Video TIDAK ditemukan di: {VIDEO_INPUT_DIR}")

if os.path.exists(GROUND_TRUTH_FILE):
    print(f"File Transkrip Manual ditemukan.")
else:
    print(f"File Transkrip Manual tidak ditemukan di: {GROUND_TRUTH_FILE}")

try:
    model = whisper.load_model("small")
    print("Model Whisper berhasil dimuat ke dalam sistem.")
except Exception as e:
    print(f"Gagal memuat model: {e}")

## Processing Functions

In [ ]:
def convert_video_to_audio(video_path, audio_path):
    try:
        video_clip = mp.VideoFileClip(video_path)
        video_clip.audio.write_audiofile(audio_path, codec='pcm_s16le', verbose=False, logger=None)
        video_clip.close()
        return True
    except Exception:
        return False

def transcribe_audio(audio_path):
    try:
        technical_prompt = (
            "Transcribe strictly in English. Context: Machine Learning interview. "
            "Keywords: TensorFlow, Scikit-learn, CNN, Dropout, Overfitting, Transfer Learning. "
            "Do not include filler words like umm, uh, ah."
        )

        result = model.transcribe(
            audio_path,
            fp16=False,
            language="en",
            initial_prompt=technical_prompt
        )
        return result["text"].strip()
    except Exception as e:
        print(f"[ERROR] Transkripsi: {e}")
        return ""

def remove_fillers(text):
    #Daftar kata filler yang dihapus
    fillers = [
        r"\bum\b", r"\buh\b", r"\buhh\b", r"\bah\b", r"\ber\b", r"\bhmm\b",
        r"\bmhm\b", r"\buh-huh\b", r"\bokay\b",
        r"\byou know\b", r"\bi mean\b", r"\bkind of\b", r"\bsort of\b",
        r"\bso\b", r"\blike\b", r"\byeah\b", r"\bright\b",
    ]

    clean_text = text.lower()
    for filler in fillers:
        clean_text = re.sub(filler, "", clean_text)

    #Hapus spasi ganda
    clean_text = re.sub(r'\s+', ' ', clean_text).strip()
    return clean_text

def calculate_metrics(reference_text, hypothesis_text):
    if not reference_text or not hypothesis_text:
        return {"wer": 1.0, "accuracy": 0.0}

    #Bersihkan tanda baca dasar
    transformation = jiwer.Compose([
        jiwer.ToLowerCase(),
        jiwer.RemovePunctuation(),
        jiwer.RemoveMultipleSpaces(),
        jiwer.Strip(),
    ])

    ref_basic = transformation(reference_text)
    hyp_basic = transformation(hypothesis_text)

    #Hapus filler words
    ref_clean = remove_fillers(ref_basic)
    hyp_clean = remove_fillers(hyp_basic)

    #Hitung Akurasi
    wer_score = jiwer.wer(ref_clean, hyp_clean)
    accuracy = max(0, 1 - wer_score) * 100

    return {"wer": wer_score, "accuracy": round(accuracy, 2)}

## Processing Pipeline & Evaluation

In [ ]:
TRANSCRIPT_DIR = os.path.join(BASE_DIR, "Transkrip_Manual")

#CONVERT VIDEO KE AUDIO
video_files = [f for f in os.listdir(VIDEO_INPUT_DIR) if f.lower().endswith(('.mp4', '.avi', '.webm'))]
print(f"\n[STEP 1] Cek Video: {len(video_files)} file ditemukan.")

for video in tqdm(video_files, desc="Converting Videos"):
    v_path = os.path.join(VIDEO_INPUT_DIR, video)
    a_path = os.path.join(AUDIO_OUTPUT_DIR, os.path.splitext(video)[0] + ".wav")

    if not os.path.exists(a_path):
        convert_video_to_audio(v_path, a_path)

#PROSES SEMUA AUDIO
all_audio_files = []

for f in os.listdir(AUDIO_OUTPUT_DIR):
    if f.lower().endswith('.wav'):
        all_audio_files.append(os.path.join(AUDIO_OUTPUT_DIR, f))

#Hilangkan duplikat path
all_audio_files = list(set(all_audio_files))

print(f"\n[STEP 2] Total File Audio Siap Proses: {len(all_audio_files)} file")

#LOOP TRANSKRIPSI & EVALUASI
processing_results = []
total_accuracy = 0
count_evaluated = 0

for audio_path in tqdm(all_audio_files, desc="AI Transcribing"):
    filename = os.path.basename(audio_path)
    base_name = os.path.splitext(filename)[0]

    pred_text = transcribe_audio(audio_path)

    metrics = {"accuracy": 0.0}
    truth_text = "N/A"

    txt_path = os.path.join(TRANSCRIPT_DIR, base_name + ".txt")

    if os.path.exists(txt_path):
        try:
            with open(txt_path, 'r', encoding='utf-8') as f:
                truth_text = f.read().strip()
            #Hitung akurasi
            metrics = calculate_metrics(truth_text, pred_text)
            total_accuracy += metrics["accuracy"]
            count_evaluated += 1
        except: pass

    #Simpan hasil ke list
    processing_results.append({
        "filename": filename,
        "prediction": pred_text,
        "ground_truth": truth_text[:100], # Preview aja
        "accuracy": metrics["accuracy"]
    })

print("-" * 50)
df_results = pd.DataFrame(processing_results)

if count_evaluated > 0:
    avg_accuracy = total_accuracy / count_evaluated
else:
    avg_accuracy = 0.0

print(f"Total Data Diproses       : {len(processing_results)}")
print(f"Data dengan Kunci Jawaban : {count_evaluated}")
print(f"Rata-rata Akurasi         : {avg_accuracy:.2f}%")

if avg_accuracy >= 90:
    print("STATUS: LULUS (Akurasi >= 90%)")
else:
    print("STATUS: BELUM LULUS")

df_results[["filename", "accuracy"]]

# AI Assessment & JSON Generation

In [ ]:
KEYWORD_DB = {
    #Soal Machine Learning
    1: ["challenge", "overcame", "team", "disagreement", "listen", "meeting", "risk"],
    2: ["transfer learning", "vgg", "resnet", "mobilenet", "efficient", "keras", "accuracy"],
    3: ["model", "accuracy", "efficiency", "layers", "dense", "dropout", "smote", "imbalanced"],
    4: ["dropout", "overfitting", "training", "layer", "rate", "neural network", "epoch"],
    5: ["cnn", "convolutional", "pooling", "flatten", "filters", "image", "classification", "conv2d"],
    6: ["background", "technology", "solve", "problems", "future", "career", "engineer"],
    7: ["python", "pandas", "scikit-learn", "tensorflow", "pytorch", "preprocessing", "tools"],
    8: ["machine learning", "learn", "data", "patterns", "predictions", "examples", "adapt"],
    9: ["debug", "error", "check", "data", "hyperparameters", "learning rate", "batch size"],

    #Soal Arsitektur dan General
    10: ["curious", "patient", "disciplined", "problem-solving", "adapt", "learn", "quality"],
    11: ["creativity", "engineering", "sustainable", "community", "design", "impact"],
    12: ["autocad", "sketchup", "revit", "bim", "rendering", "lumion", "3d", "software"],
    13: ["purpose", "users", "environment", "sketches", "zoning", "flow", "light"],
    14: ["feedback", "criticism", "collaborative", "improve", "revise", "iteration", "open-minded"],
    15: ["passion", "dedication", "creativity", "analytical", "team", "value", "growth"],
    16: ["resume", "projects", "capstone", "internship", "experience", "python", "sql"],
    17: ["challenging", "problem", "speed", "optimization", "inference", "deploy", "solution"],
    18: ["traveloka", "user", "product", "blog", "team", "scale", "impact"],
    19: ["teamwork", "disagreement", "listen", "meeting", "compromise", "result"],
    20: ["simple", "analogy", "explain", "concept", "non-technical", "understand"]
}

def extract_id_from_filename(filename):
    numbers = re.findall(r'\d+', filename)
    if numbers:
        return int(numbers[0])
    return 999

def assess_answer_quality(text, question_id):
    """Menilai jawaban (0-4)"""
    #Validasi input kosong
    if not text or len(text) < 10:
        return 0, "Unanswered"

    target_keywords = KEYWORD_DB.get(question_id, [])
    #Fallback jika ID tidak ada
    if not target_keywords:
        target_keywords = ["experience", "project", "learn", "team", "problem", "solution"]

    text_lower = text.lower()
    hit_count = sum(1 for k in target_keywords if k in text_lower)
    word_count = len(text.split())

    score = 2
    reason = "General Response with Limited Details."

    if word_count > 25:
        if hit_count >= 3:
            score = 4
            reason = "Comprehensive and Very Clear Response (Contains key technical terms)."
        elif hit_count >= 1:
            score = 3
            reason = "Specific Explanation with Basic Understanding."

    return score, reason

def generate_final_report(results_data):
    final_output = {
        "success": True,
        "data": {
            "id": 131,
            "candidate": {
                "name": "Hafiz Putra Mahesta",
                "email": "phafiz726@gmail.com",
                "photoUrl": "https://path/to/photo.png"
            },
            "assessorProfile": {
                "id": 47,
                "name": "AI Assessment System",
                "photoUrl": "https://path/to/system_logo.png"
            },
            "reviewedAt": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "decision": "PASSED",
            "scoresOverview": {"project": 100, "interview": 0, "total": 0},
            "reviewChecklistResult": {
                "project": [],
                "interviews": {
                    "minScore": 0, "maxScore": 4, "scores": []
                }
            },
            "Overall notes": "Automated assessment by AI System based on Whisper transcription."
        }
    }

    total_interview_score = 0
    count_questions = 0

    sorted_results = sorted(results_data, key=lambda x: extract_id_from_filename(x['filename']))

    for item in sorted_results:
        filename = item['filename']
        q_id = extract_id_from_filename(filename)

        #Penilaian
        score, reason = assess_answer_quality(item['prediction'], q_id)

        #TRANSKRIP LENGKAP
        full_transcript = item['prediction']

        checklist_item = {
            "id": q_id,
            "score": score,
            "reason": reason,
            "transcript_preview": full_transcript
        }

        final_output["data"]["reviewChecklistResult"]["interviews"]["scores"].append(checklist_item)

        #Hitung Total
        if 1 <= q_id <= 20:
            total_interview_score += score
            count_questions += 1

    #Finalisasi Skor
    if count_questions > 0:
        max_possible_score = count_questions * 4
        interview_final_score = (total_interview_score / max_possible_score) * 100
    else:
        interview_final_score = 0

    final_output["data"]["scoresOverview"]["interview"] = round(interview_final_score, 2)

    project_score = 100
    total_final = (project_score + interview_final_score) / 2
    final_output["data"]["scoresOverview"]["total"] = round(total_final, 2)

    final_output["data"]["decision"] = "PASSED" if total_final >= 75 else "Need Human Review"
    return final_output

if 'processing_results' in locals() and processing_results:
    data_to_process = processing_results
    print("Menggunakan Data Transkripsi ASLI.")
else:
    print("Data asli tidak ditemukan. Menggunakan Dummy Data untuk Demo.")
    data_to_process = [
        {"filename": "interview_question_1.wav", "prediction": "Dummy answer 1 full text." * 10},
        {"filename": "interview_question_10.wav", "prediction": "Dummy answer 10 full text." * 10},
        {"filename": "interview_question_2.wav", "prediction": "Dummy answer 2 full text." * 10}
    ]

json_report = generate_final_report(data_to_process)

print(json.dumps(json_report, indent=2))

output_json_path = os.path.join(BASE_DIR, "final_assessment_result.json")
with open(output_json_path, "w") as f:
    json.dump(json_report, f, indent=2)

# Interface Streamlit

In [ ]:
if os.path.exists("app.py"):
    os.remove("app.py")

app_code = """import streamlit as st
import whisper
import os
import json
import re
from datetime import datetime
import moviepy.editor as mp

# --- KONFIGURASI HALAMAN ---
st.set_page_config(page_title="AI Interview Assessor", layout="wide")

# --- 1. LOAD MODEL WHISPER (CACHED) ---
# Kita gunakan @st.cache_resource agar model tidak di-load ulang setiap kali klik tombol
@st.cache_resource
def load_whisper_model():
    # Load model 'small' (sesuai yang kita pakai di notebook)
    return whisper.load_model("small")

try:
    model = load_whisper_model()
except Exception as e:
    st.error(f"Gagal memuat model Whisper: {e}")

# --- 2. FUNGSI PENDUKUNG (COPY DARI NOTEBOOK) ---
# Fungsi Konversi Video -> Audio
def convert_video_to_audio(video_path, audio_path):
    try:
        video_clip = mp.VideoFileClip(video_path)
        video_clip.audio.write_audiofile(audio_path, codec='pcm_s16le', verbose=False, logger=None)
        video_clip.close()
        return True
    except: return False

# Fungsi Transkripsi
def transcribe(audio_path):
    technical_prompt = (
        "Transcribe strictly in English. Context: Machine Learning & Architecture interview. "
        "Keywords: TensorFlow, CNN, Dropout, Overfitting, Scikit-learn, AutoCAD, Revit, BIM. "
        "Do not include filler words."
    )
    result = model.transcribe(audio_path, fp16=False, language="en", initial_prompt=technical_prompt)
    return result["text"].strip()

# Fungsi Penilaian (Simple Rule-Based)
KEYWORD_DB = {
    "ml": ["tensorflow", "model", "accuracy", "layer", "training", "data", "learning", "cnn"],
    "general": ["challenge", "team", "project", "solution", "role", "experience"]
}

def assess_answer(text):
    if len(text) < 10: return 0, "Unanswered / Too Short"

    # Hitung kata kunci (Gabungan ML & General)
    keywords = KEYWORD_DB["ml"] + KEYWORD_DB["general"]
    text_lower = text.lower()
    hit_count = sum(1 for k in keywords if k in text_lower)
    word_count = len(text.split())

    if word_count > 30 and hit_count >= 3:
        return 4, "Comprehensive and Very Clear Response (Contains technical terms)."
    elif word_count > 20 and hit_count >= 1:
        return 3, "Specific Explanation with Basic Understanding."
    else:
        return 2, "General Response with Limited Details."

# --- 3. TAMPILAN SIDEBAR (INPUT DATA PENGGUNA) ---
with st.sidebar:
    st.title("Candidate Profile")
    cand_name = st.text_input("Full Name", "Hafiz Putra Mahesta")
    cand_email = st.text_input("Email", "hafiz@dicoding.com")
    cand_role = st.selectbox("Position Applied", ["Machine Learning Engineer", "AI Architect", "Data Scientist"])

    st.divider()
    st.info("Upload video wawancara di panel utama untuk memulai analisis.")

# --- 4. TAMPILAN UTAMA ---
st.title("AI-Powered Interview Assessment")
st.markdown("Unggah video wawancara kandidat, dan AI akan melakukan transkripsi serta penilaian otomatis.")

# Input File
uploaded_file = st.file_uploader("Upload Video Interview (.mp4, .mov, .avi)", type=["mp4", "mov", "avi", "webm"])

if uploaded_file is not None:
    # Tampilkan Video
    st.video(uploaded_file)

    # Tombol Proses
    if st.button("Start AI Analysis", type="primary"):

        # A. Simpan File Sementara
        os.makedirs("temp_upload", exist_ok=True)
        video_path = os.path.join("temp_upload", uploaded_file.name)
        audio_path = video_path.replace(".mp4", ".wav").replace(".webm", ".wav")

        with open(video_path, "wb") as f:
            f.write(uploaded_file.getbuffer())

        # B. Proses AI (Loading Bar)
        with st.status("Sedang memproses...", expanded=True) as status:

            st.write("Mengonversi Video ke Audio...")
            convert_video_to_audio(video_path, audio_path)

            st.write("Whisper AI sedang mendengarkan & mentranskrip...")
            transcript_text = transcribe(audio_path)

            st.write("Melakukan penilaian otomatis...")
            score, reason = assess_answer(transcript_text)

            status.update(label="Analisis Selesai!", state="complete", expanded=False)

        # C. Tampilkan Hasil
        st.divider()
        st.subheader("Analysis Result")

        c1, c2 = st.columns([2, 1])

        with c1:
            st.markdown("### Transcript")
            st.info(transcript_text)

        with c2:
            st.markdown("### Assessment")
            if score >= 3:
                st.success(f"**Score: {score}/4**")
            else:
                st.warning(f"**Score: {score}/4**")
            st.caption(f"Reason: {reason}")

        # D. Generate JSON Output (Sesuai Payload)
        final_json = {
            "success": True,
            "data": {
                "candidate": {
                    "name": cand_name,
                    "email": cand_email,
                    "role": cand_role
                },
                "assessorProfile": {
                    "id": 47,
                    "name": "AI Assessment System"
                },
                "reviewedAt": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                "decision": "PASSED" if score >= 3 else "REVIEW NEEDED",
                "scoresOverview": {
                    "total": (score/4)*100
                },
                "reviewChecklistResult": {
                    "interviews": {
                        "scores": [{
                            "score": score,
                            "reason": reason,
                            "transcript_preview": transcript_text
                        }]
                    }
                }
            }
        }

        # Tombol Download
        json_str = json.dumps(final_json, indent=2)
        st.download_button(
            label="Download Official JSON Report",
            data=json_str,
            file_name=f"assessment_{cand_name.replace(' ', '_')}.json",
            mime="application/json"
        )
"""

with open("app.py", "w") as f:
    f.write(app_code)

print("File app.py berhasil diperbarui ke Versi Interaktif!")

## Run App

In [ ]:
#Token Ngrok
NGROK_AUTH_TOKEN = "361WrbAMOsmOyORYgVMRd9pI2Q9_3LQJ8vzT92U1wmunBpcJJ"

#Mematikan process lama
!pkill -9 streamlit
!pkill -9 ngrok

#Setup Ngrok Manual
if not os.path.exists("ngrok"):
    !wget -q -c -nc https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
    !tar -xvzf ngrok-v3-stable-linux-amd64.tgz
    !chmod +x ngrok

conf.get_default().ngrok_path = "./ngrok"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

#Menjalankan Streamlit
print("Menjalankan Streamlit...")
get_ipython().system_raw('streamlit run app.py &')

time.sleep(5)
try:
    public_url = ngrok.connect(8501).public_url
    print(f"\n KLIK LINK: {public_url}")
except Exception as e:
    print(f"Error: {e}")